In [1]:
from bs4 import BeautifulSoup
import requests
import time
from random import randint
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import os
import pickle

In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36'}

In [102]:
chromedriver = "/Applications/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver
driver = webdriver.Chrome(chromedriver)

In [103]:
def scroll_bottom():
    # Vivino specifed crawl delay
    counter = 0

    while True:
        counter += 1 
        
        if counter >= 1000:
            break
    
        # Get scroll height
        last_height = driver.execute_script("return document.body.scrollHeight")

        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(10)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:

            # try again (can be removed)
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(10)

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")

            # check if the page height has remained the same
            if new_height == last_height:
                # if so, you are done
                break
            # if not, move on to the next loop
            else:
                last_height = new_height
                continue

In [104]:
def get_wine_url(base_url = 'https://www.vivino.com/explore?e=eJzLLbI1VMvNzLM1UMtNrLA1NTBQS660DQ1WSwYSLmoFQNn0NNuyxKLM1JLEHLX8JNuixJLMvPTi-OT80rwStXzblNTiZLXykuhYoFIwZQSlTCC0MYQyh1AmasW2yYkA-pkm0g%3D%3D&cart_item_source=nav-explore'):
    driver.get(base_url)
    
    #scroll
    scroll_bottom()
    
    #get links
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    #wine_page_links = [link.get('href').strip() for link in soup.find_all('a', class_='anchor__anchor--2QZvA')]
    wine_page_links = []
    for link in soup.find_all('a', class_='anchor__anchor--2QZvA'):
        if "year=" in link.get('href').strip() and "cart_item_source" in link.get('href').strip():
            wine_page_links.append(link.get('href').strip())        
        else:
            pass
    
    return wine_page_links

In [105]:
wine_page_links = get_wine_url()

KeyboardInterrupt: 

In [106]:
len(wine_page_links)

1449

In [ ]:
wine_page_links[:10]


In [12]:
def parse_ratings(soup):
    rating_raw = soup.find_all("div", class_=lambda value: value and value.startswith("vivinoRatingWide"))[0]
    rating_clean = rating_raw.text.split()[0]
    return rating_clean


In [13]:
def parse_winery(soup):
    winery_raw = soup.find(class_="winery")
    winery_clean = winery_raw.text.strip('\n')
    return winery_clean

In [14]:
def parse_wine_name(soup):
    name_raw = soup.find(class_="vintage")
    name_raw2 = name_raw.text.split()[:-1]
    name_clean = ' '.join(name_raw2)
    return name_clean

In [15]:
def parse_wine_vintage(soup):
    vintage_raw = soup.find(class_="vintage")
    vintage_clean = vintage_raw.text.split()[-1]
    return vintage_clean

In [16]:
def parse_wine_type(soup):
    type_raw = soup.find(class_="wineLocationHeader__wineType--14nrC")
    type_clean = type_raw.text
    return type_clean

In [17]:
def parse_wine_region(soup):
    region_raw = soup.find(class_="anchor__anchor--3DOSm wineLocationHeader__region--1cbip")
    region_clean = region_raw.text
    return region_clean

In [18]:
def parse_wine_country(soup):
    country_raw = soup.find(class_="anchor__anchor--3DOSm wineLocationHeader__country--1RcW2")
    country_clean = country_raw.text
    return country_clean

In [19]:
def parse_number_reviews(soup):   
    reviews_raw = soup.find(class_="vivinoRatingWide__basedOn--s6y0t")
    reviews_clean = reviews_raw.text[:-8]
    return reviews_clean

In [71]:
def parse_avg_price(soup):
    price = soup.find("span", class_=lambda value: value and value.startswith("purchaseAvailability"))
    return price.text[1:]
    

In [107]:
def scrape_wine_info():
    avg_rating_list = []
    winery_list = []
    name_list = []
    vintage_list =[]
    type_list = []
    region_list = []
    country_list = []
    num_reviews_list = []
    avg_price_list = []

    base_url = 'https://www.vivino.com'
    chromedriver = "/Applications/chromedriver"
    os.environ["webdriver.chrome.driver"] = chromedriver
    driver = webdriver.Chrome(chromedriver)
    
    for page in wine_page_links:
        time.sleep(10)
        driver.get(base_url + page)
        soup = BeautifulSoup(driver.page_source, 'html.parser')      

        try:
            rating = parse_ratings(soup)
            avg_rating_list.append(rating)
        except:
            avg_rating_list.append('error')
            
        try:
            winery = parse_winery(soup)
            winery_list.append(winery)
        except:
            winery_list.append('error')
        
        try:
            name = parse_wine_name(soup)
            name_list.append(name)
        except:
            name_list.append('error')
            
        try:
            year = parse_wine_vintage(soup)
            vintage_list.append(year)
        except:
            vintage_list.append('error')
            
        try:
            wine_type = parse_wine_type(soup)
            type_list.append(wine_type)
        except:
            type_list.append('error') 
            
        try:
            region = parse_wine_region(soup)
            region_list.append(region)
        except:
            region_list.append('error')   
        
        try:
            country = parse_wine_country(soup)
            country_list.append(country)
        except:
            country_list.append('error')
        
        try:
            num_reviews = parse_number_reviews(soup)
            num_reviews_list.append(num_reviews)
        except:
            num_reviews_list.append('error')
            
        try:
            avg_price = parse_avg_price(soup)
            avg_price_list.append(avg_price)
        except:
            avg_price_list.append('error')
         
            
    features = ('Average Rating', 'Winery', 'Wine Name', 'Vintage',
                'Type', 'Region', 'Country', 'Number of Reviews',
               'Average Price')
    df = pd.DataFrame(list(zip(avg_rating_list, winery_list, name_list, vintage_list, type_list, region_list, 
                               country_list, num_reviews_list, avg_price_list)), columns =features) 
    
    return df

In [108]:
df = scrape_wine_info()

In [109]:
df.head()

,Average Rating,Winery,Wine Name,Vintage,Type,Region,Country,Number of Reviews,Average Price
0,4.298089,Veuve Clicquot,Brut (Carte Jaune) Champagne,N.V.,Sparkling wine,Champagne,France,98089,49.99
1,4.186996,Moët & Chandon,Impérial Brut Champagne,N.V.,Sparkling wine,Champagne,France,86996,error
2,3.767923,Casal Garcia,Vinho Verde Branco,N.V.,White wine,Vinho Verde,Portugal,67923,error
3,4.352852,Farnese,Edizione Cinque Autoctoni,N.V.,Red wine,Abruzzo,Italy,52852,error
4,3.847404,Piccini,Memoro Rosso,N.V.,Red wine,Toscana,Italy,47404,10.99


In [110]:
#pickle
with open('wine_data_long.pkl', 'wb') as picklefile:
    pickle.dump(df, picklefile)

In [111]:
with open('wine_data_long.pkl', 'rb') as picklefile: 
    wine_data = pickle.load(picklefile)

In [112]:
wine_data

,Average Rating,Winery,Wine Name,Vintage,Type,Region,Country,Number of Reviews,Average Price
0,4.298089,Veuve Clicquot,Brut (Carte Jaune) Champagne,N.V.,Sparkling wine,Champagne,France,98089,49.99
1,4.186996,Moët & Chandon,Impérial Brut Champagne,N.V.,Sparkling wine,Champagne,France,86996,error
2,3.767923,Casal Garcia,Vinho Verde Branco,N.V.,White wine,Vinho Verde,Portugal,67923,error
3,4.352852,Farnese,Edizione Cinque Autoctoni,N.V.,Red wine,Abruzzo,Italy,52852,error
4,3.847404,Piccini,Memoro Rosso,N.V.,Red wine,Toscana,Italy,47404,10.99
...,...,...,...,...,...,...,...,...,...
1444,3.63780,Louis Jadot,Bourgogne Pinot Noir,2015,Red wine,Bourgogne,France,3780,error
1445,4.03780,Enzo Bartoli,Barbera d’Asti Superiore,2017,Red wine,Barbera d'Asti,Italy,3780,16.80
1446,3.63780,Dark Horse,Cabernet Sauvignon,2016,Red wine,California,United States,3780,error
1447,4.63778,Vega Sicilia,Valbuena 5º,2013,Red wine,Ribera del Duero,Spain,3778,error
